In [25]:
from pathlib import Path
import pandas as pd
import numpy as np
import torch
from src.models.particle_filter import ParticleFilterMLP, ParticleFilterModel
from src.models.normal import NormalDegradationModel as NModel
import imageio
import matplotlib.pyplot as plt

In [26]:
data_name = "DS03"
perform_name = "SmLPC"
device = "cpu"  # or "cuda"

experiment_dir = Path('experiments')/data_name
states_dir = experiment_dir/'states'

## Import training data

In [27]:
hi_df = pd.read_csv(experiment_dir/'hidata_dev.csv')

### get only degraded data

In [28]:
hi_df = hi_df[hi_df['hs']==0]
del hi_df['hs']

In [29]:
perform_alias = [col for col in hi_df.columns if col not in ['unit','cycle']]
units = hi_df['unit'].unique().tolist()

In [30]:
performs_train = {name: 
    {int(unit): hi_df[hi_df['unit']==unit][name].values for unit in units} 
    for name in perform_alias
}
time_train = {int(unit): hi_df[hi_df['unit']==unit]['cycle'].values for unit in units}

## Prepare data

In [31]:
units = list(time_train.keys())
units

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [32]:
eval_data = {}
for unit in units:
    t_data = time_train[unit]
    s_data = performs_train[perform_name][unit]
    eval_data[unit]=torch.tensor(np.stack([t_data, s_data],axis=1),dtype=torch.float32).to(device)


## Load component (base) models

In [33]:
train_degmodels = {}
for eval_unit in units:
    degmodels=[]
    for unit, perform in performs_train[perform_name].items():
        if unit == eval_unit:
            continue
        best_model = NModel()
        best_model.load_state_dict(
            torch.load(states_dir /perform_name/f'unit_{unit}'/ "best_model.pt", map_location=device)
        )
        best_model.to(device)
        degmodels.append(best_model)
    train_degmodels[eval_unit] = degmodels

## Load PF-net

In [34]:
# 1. Recreate the network with the SAME architecture
net = ParticleFilterMLP(state_dim=5,hidden_dims=[128, 128, 32])

# 2. Load weights
net.load_state_dict(
    torch.load(
        states_dir / perform_name / "best_pf_net.pt",
        map_location=device,
    )
)

# 3. Move to device
net = net.to(device)

# 4. Set mode explicitly
net.eval()    # or net.train()

ParticleFilterMLP(
  (net): Sequential(
    (0): Linear(in_features=2, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=6, bias=True)
  )
)

In [ ]:
n_particles = 1800
multiply_scale = 0.05
start_idx = 5

## Run Particle Filter with (leraned net)

Save video

In [36]:
t_grid = np.linspace(0.1, 100, 400)
s_grid = np.linspace(0.0, 1.0, 300)


for eval_unit in units:
	degmodels = train_degmodels[eval_unit]
	t_data = eval_data[eval_unit][:, 0]
	s_data = eval_data[eval_unit][:, 1]

	pf = ParticleFilterModel(
		base_models=degmodels,
		net=net,
		max_life=100.0,
		n_particles=n_particles,
		multiply_scale=multiply_scale,
		name=perform_name,
	).eval()

	frames = []
	for k in range(start_idx, len(t_data)):
		pf.step(
			t_obs=t_data[:k],
			s_obs=s_data[:k],
		)
  
		# --- render frame ---
		fig, ax = plt.subplots(figsize=(10, 8))
		pf.plot(
	    ax=ax,
	    t_grid=t_grid,
	    s_grid=s_grid,
	    t_obs=t_data[:k].numpy(),
	    s_obs=s_data[:k].numpy(),
	    title=f"PF prediction | step {k}",
	)
		# create frame
		fig.canvas.draw()
		frame = np.asarray(fig.canvas.renderer.buffer_rgba())
		plt.close(fig)
		frames.append(frame)
  
	## Save video
	video_path = states_dir / perform_name / f"pf_eval{eval_unit}.mp4"

	with imageio.get_writer(video_path, fps=8, macro_block_size=1) as writer:
		for frame in frames:
			writer.append_data(frame)

	print(f"🎬 Video saved to {video_path}")


/var/folders/hl/2v5kr9994fl_j6sv_4mt0bn00000gn/T/ipykernel_66603/3543694453.py:37: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.draw()
Python(87671) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


🎬 Video saved to experiments/DS03/states/SmLPC/pf_eval1.mp4


Python(634) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


🎬 Video saved to experiments/DS03/states/SmLPC/pf_eval2.mp4


Python(11466) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


...

🎬 Video saved to experiments/DS03/states/SmLPC/pf_eval7.mp4


Python(51136) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


🎬 Video saved to experiments/DS03/states/SmLPC/pf_eval8.mp4


Python(52414) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


🎬 Video saved to experiments/DS03/states/SmLPC/pf_eval9.mp4
